In [1]:
import pandas as pd

In [2]:
# Import source data and define as dataframes
schools_path = "Resources/schools_complete.csv"
students_path = "Resources/students_complete.csv"

school_data = pd.read_csv(schools_path)
student_data = pd.read_csv(students_path)

In [3]:
# Determine per student and test whether they passed each test and overall (both tests), this will be used throughout
student_data["math_pass"] = student_data["math_score"] >= 70
student_data["reading_pass"] = student_data["reading_score"] >= 70
student_data["both_pass"] = (student_data["math_score"] >= 70) & (student_data["reading_score"] >= 70)

#### District Summary Report - Construction

In [4]:
# Define the simple analysis to go into the summary dataframe:
    # Count of schools
total_schools = len(school_data["school_name"].unique())
    # Count of students
total_students = student_data["Student ID"].count()
    # Sum of all school budgets
total_budget = school_data["budget"].sum()
    # Average of all student math scores
average_math = student_data["math_score"].mean()
    # Average of all student reading scores
average_read = student_data["reading_score"].mean()

In [5]:
# Determine for all students the pass rate for math, reading, and overall for the summary dataframe
ps_rate_math = (student_data["math_pass"].sum() / total_students) * 100
ps_rate_read = (student_data["reading_pass"].sum() / total_students) * 100
ps_rate_both = (student_data["both_pass"].sum() / total_students) * 100

In [6]:
# Construct the district summary dataframe, help Pandas with brackets on a single value
dist_summary_df = pd.DataFrame({"Total Schools" : [total_schools],
                   "Total Students" : "{:,}".format(total_students),
                   "Total Budget" : "$ {:,.2f}".format(total_budget),
                   "Average Math Score" : "{:.2f}".format(average_math),
                   "Average Reading Score" : "{:.2f}".format(average_read),
                   "Math Pass Rate": "{:.2f}%".format(ps_rate_math),
                   "Reading Pass Rate": "{:.2f}%".format(ps_rate_read),
                   "Overall Pass Rate": "{:.2f}%".format(ps_rate_both)})

### District Summary - Report

In [7]:
dist_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Math Pass Rate,Reading Pass Rate,Overall Pass Rate
0,15,"39,170","$ 24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


#### School Summary - Construction

In [8]:
# Take only the columns we need per student, i.e. nothing is asking for age or gender
students_sanitized = student_data[["Student ID","grade","school_name","reading_score","math_score",
                                   "math_pass","reading_pass","both_pass"]]
# Calculate budget per student and place into the school data table
school_data["per_student_budget"] = school_data["budget"] / school_data["size"]
## Merge the tables as we will now be looking at specific data on students per school
district_students = pd.merge(students_sanitized, school_data, on="school_name",how="left")

In [9]:
# Group by school and calculate the statistics needed per school
groupby_school = district_students.groupby(["school_name"])
    # Average reading score
school_mean_read = groupby_school["reading_score"].mean()
    # Average math score
school_mean_math = groupby_school["math_score"].mean()
    # Math pass rate
school_pass_math = (groupby_school["math_pass"].sum() / groupby_school["Student ID"].count()) * 100
    # Reading pass rate
school_pass_read = (groupby_school["reading_pass"].sum() / groupby_school["Student ID"].count()) * 100
    # Overall pass rate
school_pass_both = (groupby_school["both_pass"].sum() / groupby_school["Student ID"].count()) * 100

In [10]:
# Construct the dataframe and merge in the data that did not need to be calculated
school_summary_df = pd.DataFrame({"Average Math Score": school_mean_math,
                                  "Average Reading Score": school_mean_read,
                                  "Math Pass Rate": school_pass_math,
                                  "Reading Pass Rate": school_pass_read,
                                  "Overall Pass Rate": school_pass_both})
school_summary_df = pd.merge(school_data[["school_name","type","size","budget","per_student_budget"]],
                             school_summary_df,on="school_name")

In [11]:
# Rename the columns to explain better
school_summary_df = school_summary_df.rename(columns={"school_name":"School Name",
                                                      "type":"School Type",
                                                      "size":"Total Students",
                                                      "budget":"Total School Budget",
                                                      "per_student_budget":"Per Student Budget"})
# Save a copy to use later, before we change the numbers into more readable formats as strings
school_summary_df_unf = school_summary_df.copy()

In [12]:
# Format data
school_summary_df["Total Students"] = school_summary_df["Total Students"].map("{:,}".format)
school_summary_df["Total School Budget"] = school_summary_df["Total School Budget"].map("$ {:,.2f}".format)
school_summary_df["Per Student Budget"] = school_summary_df["Per Student Budget"].map("$ {:,.2f}".format)
school_summary_df["Average Math Score"] = school_summary_df["Average Math Score"].map("{:,.2f}".format)
school_summary_df["Average Reading Score"] = school_summary_df["Average Reading Score"].map("{:,.2f}".format)
school_summary_df["Math Pass Rate"] = school_summary_df["Math Pass Rate"].map("{:,.2f}%".format)
school_summary_df["Reading Pass Rate"] = school_summary_df["Reading Pass Rate"].map("{:,.2f}%".format)
school_summary_df["Overall Pass Rate"] = school_summary_df["Overall Pass Rate"].map("{:,.2f}%".format)

In [13]:
# Sort data by school name and replace the index value with the school name
school_summary_df = school_summary_df.sort_values("School Name")
school_summary_df.set_index("School Name", inplace=True)

### School Summary - Report

In [14]:
school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Math Pass Rate,Reading Pass Rate,Overall Pass Rate
School Name,,,,,,,,,
Bailey High School,District,"4,976","$ 3,124,928.00",$ 628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,"1,858","$ 1,081,356.00",$ 582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,"2,949","$ 1,884,411.00",$ 639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,"2,739","$ 1,763,916.00",$ 644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,"1,468","$ 917,500.00",$ 625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,"4,635","$ 3,022,020.00",$ 652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$ 248,087.00",$ 581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,"2,917","$ 1,910,635.00",$ 655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,"4,761","$ 3,094,650.00",$ 650.00,77.07,80.97,66.06%,81.22%,53.54%


#### Top Performing Schools - Construction

In [15]:
# Sort by Overall Pass Rate descending
top_perf_schools_df = school_summary_df.sort_values("Overall Pass Rate", ascending=False)

### Top Performing Schools (by Overall Passing Rate) - Report

In [16]:
top_perf_schools_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Math Pass Rate,Reading Pass Rate,Overall Pass Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$ 1,081,356.00",$ 582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,"1,635","$ 1,043,130.00",$ 638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,"1,468","$ 917,500.00",$ 625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,"2,283","$ 1,319,574.00",$ 578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$ 585,858.00",$ 609.00,83.84,84.04,94.59%,95.95%,90.54%


#### Bottom Performing Schools - Construction

In [17]:
# Sort by Overall Pass Rate ascending
low_perf_schools_df = school_summary_df.sort_values("Overall Pass Rate")

### Bottom Performing Schools (by Overall Passing Rate) - Report

In [18]:
low_perf_schools_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Math Pass Rate,Reading Pass Rate,Overall Pass Rate
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$ 2,547,363.00",$ 637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,"2,949","$ 1,884,411.00",$ 639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,"2,917","$ 1,910,635.00",$ 655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,"4,635","$ 3,022,020.00",$ 652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,"4,761","$ 3,094,650.00",$ 650.00,77.07,80.97,66.06%,81.22%,53.54%


#### (Math & Reading) Scores by Grade - Preparation

In [19]:
# Rename School Name to prep for reporting
students_sanitized = students_sanitized.rename(columns={"school_name":"School Name"})

# Isolate students by grade level
grade09scores = students_sanitized.loc[(students_sanitized["grade"] == "9th")]
grade10scores = students_sanitized.loc[(students_sanitized["grade"] == "10th")]
grade11scores = students_sanitized.loc[(students_sanitized["grade"] == "11th")]
grade12scores = students_sanitized.loc[(students_sanitized["grade"] == "12th")]

# Group students in each grade level by their school
grade09scores_gb = grade09scores.groupby(["School Name"])
grade10scores_gb = grade10scores.groupby(["School Name"])
grade11scores_gb = grade11scores.groupby(["School Name"])
grade12scores_gb = grade12scores.groupby(["School Name"])

#### Math Scores by Grade - Construction

In [20]:
# Isolate math scores from each grade and and calculate mean
math09 = grade09scores_gb["math_score"].mean().map("{:,.2f}".format)
math10 = grade10scores_gb["math_score"].mean().map("{:,.2f}".format)
math11 = grade11scores_gb["math_score"].mean().map("{:,.2f}".format)
math12 = grade12scores_gb["math_score"].mean().map("{:,.2f}".format)

# Construct summary report with the grades as columns
math_summary_df = pd.DataFrame({"9th":math09,"10th":math10,"11th":math11,"12th":math12})

### Math Scores by Grade and School - Report

In [21]:
math_summary_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


#### Reading Scores by Grade - Construction

In [22]:
# Isolate reading scores from each grade and and calculate mean
read09 = grade09scores_gb["reading_score"].mean().map("{:,.2f}".format)
read10 = grade10scores_gb["reading_score"].mean().map("{:,.2f}".format)
read11 = grade11scores_gb["reading_score"].mean().map("{:,.2f}".format)
read12 = grade12scores_gb["reading_score"].mean().map("{:,.2f}".format)

# Construct summary report with the grades as columns
read_summary_df = pd.DataFrame({"9th":read09,"10th":read10,"11th":read11,"12th":read12})

### Reading Scores by Grade and School - Report

In [23]:
read_summary_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


#### Scores by School Spending - Construction

In [24]:
## Isolate columns from unformatted dataframe into new and sort by budget to determine ranges
school_spending_df = school_summary_df_unf[["Per Student Budget","Average Math Score","Average Reading Score",
                                            "Math Pass Rate","Reading Pass Rate","Overall Pass Rate"]]
school_spending_df = school_spending_df.sort_values("Per Student Budget")

In [25]:
## Construct bins, cut and place value in dataframe
bins = [570, 600, 630, 645, 660]
group_names = ["<$599","$600-629","$630-644","$645-660"]

school_spending_df["Spending Ranges (per Student)"] = pd.cut(school_spending_df["Per Student Budget"],
                                                            bins,labels=group_names,include_lowest=True)

In [26]:
## Group by the bins and construct the grouped dataframe
school_spending_gb = school_spending_df.groupby("Spending Ranges (per Student)")
scores_by_spending = pd.DataFrame({"Average Math Score":school_spending_gb["Average Math Score"].mean(),
                                  "Average Reading Score":school_spending_gb["Average Reading Score"].mean(),
                                  "Math Pass Rate":school_spending_gb["Math Pass Rate"].mean(),
                                  "Reading Pass Rate":school_spending_gb["Reading Pass Rate"].mean(),
                                  "Overall Pass Rate":school_spending_gb["Overall Pass Rate"].mean(),})

In [27]:
## Format data
scores_by_spending["Average Math Score"] = scores_by_spending["Average Math Score"].map("{:,.2f}".format)
scores_by_spending["Average Reading Score"] = scores_by_spending["Average Reading Score"].map("{:,.2f}".format)
scores_by_spending["Math Pass Rate"] = scores_by_spending["Math Pass Rate"].map("{:,.2f}%".format)
scores_by_spending["Reading Pass Rate"] = scores_by_spending["Reading Pass Rate"].map("{:,.2f}%".format)
scores_by_spending["Overall Pass Rate"] = scores_by_spending["Overall Pass Rate"].map("{:,.2f}%".format)

### Scores by School Spending - Report

In [28]:
scores_by_spending

,Average Math Score,Average Reading Score,Math Pass Rate,Reading Pass Rate,Overall Pass Rate
Spending Ranges (per Student),,,,,
<$599,83.44,83.89,93.54%,96.46%,90.27%
$600-629,81.41,82.97,84.89%,91.67%,78.59%
$630-644,78.52,81.62,73.48%,84.39%,62.86%
$645-660,77.00,81.03,66.16%,81.13%,53.53%


#### Scores by School Size - Construction

In [29]:
## Isolate columns from unformatted dataframe into new and sort by size to determine ranges
school_size_df = school_summary_df_unf[["Total Students","Average Math Score","Average Reading Score",
                                            "Math Pass Rate","Reading Pass Rate","Overall Pass Rate"]]
school_size_df = school_size_df.sort_values("Total Students")

In [30]:
## Construct bins, cut and place value in dataframe
bins = [0, 1000, 2500, 5000]
group_names = ["Small (<1000)","Medium (1000-2500)","Large (2500-5000)"]

school_size_df["School Size"] = pd.cut(school_size_df["Total Students"],
                                                            bins,labels=group_names,include_lowest=True)

In [31]:
## Group by the bins and construct the grouped dataframe
school_size_gb = school_size_df.groupby("School Size")
scores_by_size = pd.DataFrame({"Average Math Score":school_size_gb["Average Math Score"].mean(),
                                  "Average Reading Score":school_size_gb["Average Reading Score"].mean(),
                                  "Math Pass Rate":school_size_gb["Math Pass Rate"].mean(),
                                  "Reading Pass Rate":school_size_gb["Reading Pass Rate"].mean(),
                                  "Overall Pass Rate":school_size_gb["Overall Pass Rate"].mean(),})

In [32]:
## Format data
scores_by_size["Average Math Score"] = scores_by_size["Average Math Score"].map("{:,.2f}".format)
scores_by_size["Average Reading Score"] = scores_by_size["Average Reading Score"].map("{:,.2f}".format)
scores_by_size["Math Pass Rate"] = scores_by_size["Math Pass Rate"].map("{:,.2f}%".format)
scores_by_size["Reading Pass Rate"] = scores_by_size["Reading Pass Rate"].map("{:,.2f}%".format)
scores_by_size["Overall Pass Rate"] = scores_by_size["Overall Pass Rate"].map("{:,.2f}%".format)

### Scores by School Size - Report

In [33]:
scores_by_size

,Average Math Score,Average Reading Score,Math Pass Rate,Reading Pass Rate,Overall Pass Rate
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,89.88%
Medium (1000-2500),83.36,83.89,93.64%,96.75%,90.62%
Large (2500-5000),76.96,80.97,66.55%,80.80%,53.67%


#### Scores by School Type - Construction

In [34]:
## Isolate columns from unformatted dataframe into new
school_type_df = school_summary_df_unf[["School Type","Average Math Score","Average Reading Score",
                                            "Math Pass Rate","Reading Pass Rate","Overall Pass Rate"]]

In [35]:
## Group by the type and construct the grouped dataframe
school_type_gb = school_type_df.groupby("School Type")
scores_by_type = pd.DataFrame({"Average Math Score":school_type_gb["Average Math Score"].mean(),
                                  "Average Reading Score":school_type_gb["Average Reading Score"].mean(),
                                  "Math Pass Rate":school_type_gb["Math Pass Rate"].mean(),
                                  "Reading Pass Rate":school_type_gb["Reading Pass Rate"].mean(),
                                  "Overall Pass Rate":school_type_gb["Overall Pass Rate"].mean(),})

In [36]:
## Format data
scores_by_type["Average Math Score"] = scores_by_type["Average Math Score"].map("{:,.2f}".format)
scores_by_type["Average Reading Score"] = scores_by_type["Average Reading Score"].map("{:,.2f}".format)
scores_by_type["Math Pass Rate"] = scores_by_type["Math Pass Rate"].map("{:,.2f}%".format)
scores_by_type["Reading Pass Rate"] = scores_by_type["Reading Pass Rate"].map("{:,.2f}%".format)
scores_by_type["Overall Pass Rate"] = scores_by_type["Overall Pass Rate"].map("{:,.2f}%".format)

### Scores by School Type - Report

In [37]:
scores_by_type

,Average Math Score,Average Reading Score,Math Pass Rate,Reading Pass Rate,Overall Pass Rate
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%


### Conclusions

Students generally do better on Reading than on Math; both the Pass Rate and the Average Score are higher for Reading.

Charter schools outperform District schools, despite having generally lower budgets per student; they are clustered in the lower categories, comprising all of the lowest category and none of the highest.

Smaller schools outperform larger schools, but this overlaps largely with the charter/district divide as all the charter schools are in the small and medium category and all the district schools are in the large category.